## Exercise 10 - Text classification with Tensorflow
- In this exercise you utilize the *DisneylandReviews.csv* located in data_files directory.
- This exercise has the following phases:
    - Load the data from csv file.
    - Create directory structure including sample files from the data you loaded.
    - Train your neural network with the extracted data.
    - Validate the operation of your trained model.
- Use [this example](https://hantt.pages.labranet.jamk.fi/ttc2050-material/material/10-ai-text-classification-tensorflow/) as a reference.

1 Import all the necessary libraries listed in our Tensorflow example. Read the csv file DisneylandReviews.csv into a data structure of your choice (list, dict, json...).

2 Create the directory structure presented below either by using python's os library or manually. So there should be *disney_review_data* directory which has two subdirectories: *train* and *test*. Both of those directories should then have two subdirectories: *pos* and *neg*.

```
disney_review_data
    |
    |----train
    |      |----pos
    |      |----neg
    |
    |----test
           |----pos
           |----neg
```

3 Loop through your saved data and save it as text files (.txt) into the directory structure. First 80 % of the data should go into *pos* and *neg* subdirectories under the *train* directory with the following conditions:
- pos = rating is 4 or more
- neg = rating is 2 or less

The last 20 % should go into the *pos* and *neg* subdirectories under the *test* directory using the same conditions as above. Rating value of 3 is considered to be neutral and should not be processed.

4 Use material page linked above as a reference and implement the text classification example to your notebook. Now modify it so that your Disneyland review data will be read from the directory structure you created earlier. Run the notebook and ensure that no errors are present.

5 Create some test data to verify that your model works and present the prediction results here.